In [2]:
import openai

In [3]:
import os
import builtins
from getpass import getpass
import glob

In [4]:
os.environ['OPENAI_API_KEY'] = getpass()

In [5]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [34]:
generation_prompt = """Do the following tasks referring the the use case document-
1. Analyze the use case and understand the workings of langchain library
2. Using the prompt format below, generate 10 sample prompts replacing the external non-langchain entities or resources like urls mentioned in the documentation with a similar entities or resources covering the features that langchain offers
3. The prompt format has 2 sections Instructions and Response
4. In the instructions, detail the task you are trying to accomplish using langchain
5. In the response, write the code that you would write to accomplish the task
6. Separate each prompt with a line break
7. follow the prompt format religiously

prompt format:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}

### Response:
{response}

---

"""

In [19]:
base_dir = '/Users/amir36/Documents/workspace/src/github.com/anagri/genai-finetuning-hackathon'

In [35]:
files = [
    "use-cases-md/multiple_retrieval.md",
    "use-cases-md/tagging.md",
]

In [36]:
# read all files from use-cases-md
import glob

# files = glob.glob("use-cases-md/*.md")

for file in files:
    print(f"processing use-case: {file}")
    doc = open(file, "r").read()
    # split file into name and extension
    filename = os.path.basename(file)
    name, _ = os.path.splitext(filename)
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert programmer."},
            {
                "role": "user",
                "content": "Below is the usage for the python library langchain."
                "langchain is used to interact with Large Language Models and accomplish a variety of tasks. "
                "The code below covers how to use langchain library to make API calls",
            },
            {"role": "user", "content": doc},
            {"role": "user", "content": generation_prompt},
        ],
    )
    response = completion.choices[0].message["content"]
    examples = response.split("---")
    examples = [e.strip() for e in examples]
    examples = [e for e in examples if len(e) > 0]
    print(f"generated {len(examples)} prompts")
    for i, ex in enumerate(examples):
        with builtins.open(f"{base_dir}/training-data/{name}-{i}.md", "w") as f:
            f.write(ex)

processing use-case: use-cases-md/multiple_retrieval.md
generated 2 prompts
processing use-case: use-cases-md/tagging.md
generated 9 prompts


In [1]:
start_inst = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n"

In [10]:
# read all files from training-data, if the first line of file is not as given, insert the line into the file
files = glob.glob("training-data/*.md")
for file in files:
    with open(file, "r") as f:
        first_line = f.readline()
        if first_line != start_inst:
            with open(file, "r+") as f:
                content = f.read()
                f.seek(0, 0)
                f.write(start_inst+ "\n\n" + content)
        else:
            print(f"file {file} is good")